In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 99% 1.06G/1.06G [00:15<00:00, 112MB/s]
100% 1.06G/1.06G [00:15<00:00, 73.0MB/s]


In [4]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 128

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=30,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    '/content/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/test',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 16000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [7]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [8]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    patience=3,
    factor=0.5,
    verbose=1
)


In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)


In [10]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=30,
    callbacks=[early_stop, lr_scheduler, checkpoint]
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 570ms/step - accuracy: 0.8133 - loss: 0.3988

500/500 ━━━━━━━━━━━━━━━━━━━━ 310s 608ms/step - accuracy: 0.8135 - loss: 0.3986 - val_accuracy: 0.9470 - val_loss: 0.1425 - learning_rate: 1.0000e-04
Epoch 2/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 290s 579ms/step - accuracy: 0.9306 - loss: 0.1664 - val_accuracy: 0.9350 - val_loss: 0.1433 - learning_rate: 1.0000e-04
Epoch 3/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 567ms/step - accuracy: 0.9423 - loss: 0.1525

500/500 ━━━━━━━━━━━━━━━━━━━━ 303s 606ms/step - accuracy: 0.9423 - loss: 0.1525 - val_accuracy: 0.9500 - val_loss: 0.1247 - learning_rate: 1.0000e-04
Epoch 4/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 562ms/step - accuracy: 0.9456 - loss: 0.1355

500/500 ━━━━━━━━━━━━━━━━━━━━ 298s 596ms/step - accuracy: 0.9456 - loss: 0.1355 - val_accuracy: 0.9510 - val_loss: 0.1188 - learning_rate: 1.0000e-04
Epoch 5/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 339s 630ms/step - accuracy: 0.9467 - loss: 0.1298 - val_accuracy: 0.9480 - val_loss: 0.1334 - learning_rate: 1.0000e-04
Epoch 6/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 312s 624ms/step - accuracy: 0.9440 - loss: 0.1325 - val_accuracy: 0.9540 - val_loss: 0.1192 - learning_rate: 1.0000e-04
Epoch 7/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 604ms/step - accuracy: 0.9470 - loss: 0.1278
Epoch 7: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
500/500 ━━━━━━━━━━━━━━━━━━━━ 329s 638ms/step - accuracy: 0.9470 - loss: 0.1278 - val_accuracy: 0.9560 - val_loss: 0.1208 - learning_rate: 1.0000e-04
Epoch 8/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 316s 632ms/step - accuracy: 0.9518 - loss: 0.1240 - val_accuracy: 0.9500 - val_loss: 0.1239 - learning_rate: 5.0000e-05
Epoch 9/30
500/500 ━━━━━━━━━━━━━━━━━━━━ 323s 634ms/step - 

In [11]:
loss, acc = model.evaluate(val_generator)
print(f"Validation Accuracy: {acc*100:.2f}%")

32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 573ms/step - accuracy: 0.9516 - loss: 0.1272
Validation Accuracy: 95.30%
